In [ ]:
def mask_composite(ds, start_year=0, min_lat=-90, max_lat=90, min_lon=0, max_lon=360):
    X = np.meshgrid(ds.x.data,ds.y.data)
    mask = xr.DataArray(np.sqrt(X[0]**2 + X[1]**2) <= 10, dims=['x', 'y'])
    
    idx_region = (ds.snap_time.dt.year > start_year) & (ds.snap_lat >= min_lat) & (ds.snap_lat <= max_lat) & \
    (ds.snap_lon >= min_lon) & (ds.snap_lon <= max_lon)
    
    test = ds.snap_omldamax_anom[idx_region,:,:]
    test = xr.where(abs(test)>1e10, np.nan, test)
    test = xr.where(test==0, np.nan, test)

    out_masked = xr.where(mask, test, np.nan)
    return out_masked

In [ ]:
def sel_storms_region(ds, compvar  = "omldamax", lon1 = 0, lon2 = 360, lat1 = -90, lat2 = 90):
    compvar_full = f"snap_{compvar}_anom"
    
    # if longitude region crosses prime meridian we need to re-evaluate this
    idx_reg = (ds.snap_lat > lat1) & (ds.snap_lat < lat2) & (ds.snap_lon > lon1) & (ds.snap_lon < lon2)
    
    subsamp = ds[compvar_full][idx_reg,:,:]
    subsamp = xr.where((abs(subsamp) > 1e10) | (subsamp == 0), np.nan, subsamp)
    
    X = np.meshgrid(ds.x.data,ds.y.data)
    mask = xr.DataArray(np.sqrt(X[0]**2 + X[1]**2) <= 10, dims=['x', 'y'])
    var_out = xr.where(mask, subsamp, np.nan)
    
    return var_out